In [ ]:
%load_ext autoreload
%autoreload 2
%reload_ext lab_black
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from keras.callbacks import Callback
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score

import os
import tensorflow as tf
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras import backend as K
from tensorflow.keras.layers import GaussianNoise
from utils import *

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.regularizers import l2
from tensorflow.compat.v2.keras.layers import BatchNormalization
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
from tensorflow.python.client import device_lib

In [ ]:
np.random.seed(8)
print(device_lib.list_local_devices())

# Loading images

In [ ]:
image_dir_train = "data/training/images/"
files = os.listdir(image_dir_train)
n_train = len(files)
print(f"Loading training images, images loaded: {n_train} ")
imgs_train = np.asarray(
    [load_image(image_dir_train + files[i]) for i in range(n_train)]
)
gt_dir_train = "data/training/groundtruth/"
print(f"Loading groundtruth images, images loaded: {n_train} ")
gt_imgs_train = np.asarray(
    [load_image(gt_dir_train + files[i]) for i in range(n_train)]
)

In [ ]:
imgs_train.shape

In [ ]:
gt_imgs_train.shape

In [ ]:
image_size = 400
# Patches for training
img_patches_train = [
    crop_image(imgs_train[i], image_size, image_size) for i in range(n_train)
]
gt_patches_train = [
    crop_image(gt_imgs_train[i], image_size, image_size) for i in range(n_train)
]

# Separate features and labels
X_train = np.asarray(
    [
        img_patches_train[i][j]
        for i in range(len(img_patches_train))
        for j in range(len(img_patches_train[i]))
    ]
)
Y_train = np.asarray(
    [
        gt_patches_train[i][j]
        for i in range(len(gt_patches_train))
        for j in range(len(gt_patches_train[i]))
    ]
)

In [ ]:
X_train.shape

In [ ]:
Y_train.shape

In [ ]:
image_dir_val = "data/validating/images/"
files = os.listdir(image_dir_val)
n_val = len(files)
print(f"Loading validating images, images loaded: {n_val} ")
imgs_val = np.asarray([load_image(image_dir_val + files[i]) for i in range(n_val)])
gt_dir_val = "data/validating/groundtruth/"
print(f"Loading validating groundtruth, images loaded: {n_val} ")
gt_imgs_val = np.asarray([load_image(gt_dir_val + files[i]) for i in range(n_val)])

In [ ]:
imgs_val.shape

In [ ]:
gt_imgs_val.shape

In [ ]:
image_size = 400
# Patches for validating
img_patches_val = [
    crop_image(imgs_val[i], image_size, image_size) for i in range(n_val)
]
gt_patches_val = [
    crop_image(gt_imgs_val[i], image_size, image_size) for i in range(n_val)
]

# Separate features and labels
X_val = np.asarray(
    [
        img_patches_val[i][j]
        for i in range(len(img_patches_val))
        for j in range(len(img_patches_val[i]))
    ]
)
Y_val = np.asarray(
    [
        gt_patches_val[i][j]
        for i in range(len(gt_patches_val))
        for j in range(len(gt_patches_val[i]))
    ]
)

In [ ]:
X_val.shape

In [ ]:
Y_val.shape

In [ ]:
X_train, Y_train = imag_rotation_aug(imgs_train, gt_imgs_train)

In [ ]:
X_train = np.asarray(X_train)
Y_train = np.asarray(Y_train)

In [ ]:
print(X_train.shape)
print(Y_train.shape)
n_train = Y_train.shape[0]

In [ ]:
X_val, Y_val = imag_rotation_aug(imgs_val, gt_imgs_val)

In [ ]:
X_val = np.asarray(X_val)
Y_val = np.asarray(Y_val)

In [ ]:
print(X_val.shape)
print(Y_val.shape)
n_val = Y_val.shape[0]

# Create functions to calcualte precision, recall and F-1 in the training of model

In [ ]:
def precision(y_true, y_pred):
    """Compute the Precision for the batch.
    Args:
        y_true (numpy.ndarray): the ground truth labels
        y_pred (numpy.ndarray): the predicted labels 
    Returns:
        Precision (numpy.float64): the Precision of the batch 
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def recall(y_true, y_pred):
    """Compute the Recall for the batch.
    Args:
        y_true (numpy.ndarray): the ground truth labels
        y_pred (numpy.ndarray): the predicted labels 
    Returns:
       Recall (numpy.float64): the Recal of the batch 
    """

    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


def f1(y_true, y_pred):
    """Compute the F-1 for the batch.
    Args:
        y_true (numpy.ndarray): the ground truth labels
        y_pred (numpy.ndarray): the predicted labels 
    Returns:
       F-1 (numpy.float64): the F-1 of the batch 
    """
    p = precision(y_true, y_pred)
    r = recall(y_true, y_pred)
    return 2 * ((p * r) / (p + r + K.epsilon()))

# U-Net Architecture

In [ ]:
def down(input_layer, filters, pool=True):
    """Create convloutional and residual layers to reduce dimensions.
    Args:
        input_layer (layer): input layer before convolution
        filters (numpy.int64): number of filters
    Returns:
        max_pool (layer): layer after max-pooling
        residual (connection ): connection to connect with next layers
    """
    batchnorm = BatchNormalization()(input_layer)
    conv1 = Conv2D(filters, (5, 5), padding="same", activation="relu")(batchnorm)
    residual = Conv2D(filters, (3, 3), padding="same", activation="relu")(conv1)
    if pool:
        max_pool = MaxPool2D()(residual)
        return max_pool, residual
    else:
        return residual


def up(input_layer, residual, filters):
    """Create convloutional and residual layers to increase dimensions.
    Args:
        input_layer (layer): input layer before convolution\
        residual (connection ): connection to connect with next layers
        filters (numpy.int64): number of filters
    Returns:
        conv2 (layer): convolutional layer
    """
    filters = int(filters)
    batchnorm = BatchNormalization()(input_layer)
    upsample = UpSampling2D()(batchnorm)
    upconv = Conv2D(filters, kernel_size=(2, 2), padding="same")(upsample)
    concat = Concatenate(axis=3)([residual, upconv])
    conv1 = Conv2D(filters, (5, 5), padding="same", activation="relu")(concat)
    conv2 = Conv2D(filters, (3, 3), padding="same", activation="relu")(conv1)
    return conv2


class U_NET:
    def __init__(self, shape):
        self.shape = shape
        self.model = self.initialize_U_NET(shape)

    def initialize_U_NET(self, shape):
        """Create Network Architecture.
        Args:
            shape (triplet): Size of the input layer height x width x colors (64 x 64 x 3)
        Returns:
            model (Neural Network): Architecture of the model
        """
        # Make a custom U-nets implementation.
        filters = 64
        input_layer = Input(shape=shape)
        layers = [input_layer]
        residuals = []

        # Down 1, 64
        d1, res1 = down(input_layer, filters)
        residuals.append(res1)
        filters *= 2

        # Down 2, 32
        d2, res2 = down(d1, filters)
        residuals.append(res2)
        filters *= 2

        # Down 3, 16
        d3, res3 = down(d2, filters)
        residuals.append(res3)
        filters *= 2

        # Down 4, 8
        d4, res4 = down(d3, filters)
        residuals.append(res4)
        filters *= 2

        # Up 1, 8
        up1 = up(d4, residual=residuals[-1], filters=filters / 2)
        filters /= 2

        # Up 2,  16
        up2 = up(up1, residual=residuals[-2], filters=filters / 2)
        filters /= 2

        # Up 3, 32
        up3 = up(up2, residual=residuals[-3], filters=filters / 2)
        filters /= 2

        # Up 4, 64
        up4 = up(up3, residual=residuals[-4], filters=filters / 2)

        conv_1 = Conv2D(1, 1, activation="relu")(up4)
        flaten = Flatten()(conv_1)
        batch_1 = BatchNormalization()(flaten)
        out = Dense(
            2,
            activation="sigmoid",
            kernel_regularizer=l2(0.00001),
            activity_regularizer=l2(0.00001),
        )(batch_1)
        #         flaten = Flatten()(up4)
        #         batch_1 = BatchNormalization()(flaten)
        #         dense1 = Dense(
        #             64,
        #             activation="relu",
        #             kernel_regularizer=l2(0.00001),
        #             activity_regularizer=l2(0.00001),
        #         )(flaten)
        #         batch_2 = BatchNormalization()(dense1)
        #         out = Dense(
        #             2,
        #             activation="sigmoid",
        #             kernel_regularizer=l2(0.00001),
        #             activity_regularizer=l2(0.00001),
        #         )(batch_2)

        model = Model(input_layer, out)
        model.compile(
            loss="binary_crossentropy",
            optimizer=Adam(learning_rate=0.001),
            metrics=["accuracy", recall, f1],
        )

        # Print a summary of the model to see what has been generated
        model.summary()

        return model

    def train(self):
        """Train the Model.

        Returns:
            History (History_Keras): History of the training
        """
        # Early stopping callback after 10 steps
        early_stopping = EarlyStopping(
            monitor="val_accuracy", patience=10, verbose=1, restore_best_weights=True,
        )
        # Reduce learning rate on plateau after 4 steps
        lr_callback = ReduceLROnPlateau(
            monitor="val_accuracy",
            factor=0.2,
            patience=5,
            verbose=1,
            mode="max",
            cooldown=1,
        )
        save_best = ModelCheckpoint(
            "Unet_batchnorm-{epoch:03d}-{f1:03f}-{val_f1:03f}.h5",
            save_best_only=True,
            monitor="val_accuracy",
            mode="max",
            verbose=1,
        )
        callbacks = [lr_callback, save_best, early_stopping]

        # Train the model using the previously defined functions and callbacks
        history = self.model.fit_generator(
            create_minibatch(
                X_train, Y_train, n_train, WINDOW_SIZE, BATCH_SIZE, PATCH_SIZE
            ),
            steps_per_epoch=STEPS_PER_EPOCH,
            epochs=EPOCHS,
            use_multiprocessing=False,
            workers=1,
            callbacks=callbacks,
            verbose=1,
            validation_data=create_minibatch(
                X_val, Y_val, n_val, WINDOW_SIZE, BATCH_SIZE, PATCH_SIZE
            ),
            validation_steps=STEPS_PER_EPOCH / 3,
        )
        return history

    def classify(self, X):
        """Classify Image as either road or not.
        Args:
            X (image): part of the image to classify
        Returns:
            Predictions : Predictions for each patch
        """
        # Subdivide the images into blocks with a stride and patch_size of 16
        img_patches = create_patches(X, 16, 16, padding=24)

        # Predict
        predictions = self.model.predict(img_patches)
        predictions = (predictions[:, 0] < predictions[:, 1]) * 1

        # Regroup patches into images
        return predictions.reshape(X.shape[0], -1)

    #         return group_patches(predictions, X.shape[0])

    def load(self, filename):
        """Loads Saved Model.
        Args:
           filename (string): name of the model
           
        """
        # Load the model (used for submission)
        dependencies = {
            "recall": recall,
            "f1": f1,
        }
        self.model = load_model(filename, custom_objects=dependencies)

    def save(self, filename):
        """Saves trained model.
        Args:
           filename (string): name of the model
           
        """
        self.model.save(filename)

In [ ]:
# We define parameters of the model
BATCH_SIZE = 150
WINDOW_SIZE = 64
PATCH_SIZE = 16
EPOCHS = 300
STEPS_PER_EPOCH = 100
model = U_NET(shape=(WINDOW_SIZE, WINDOW_SIZE, 3))

# Train the Model


In [ ]:
history = model.train()

In [ ]:
# plot the training loss and accuracy
plt.style.use("ggplot")
plt.figure(figsize=(15, 10))
plt.plot(history.history["loss"][1:], label="train_loss")
plt.plot(history.history["val_loss"][1:], label="val_loss")
plt.plot(history.history["accuracy"][1:], label="train_acc")
plt.plot(history.history["val_accuracy"][1:], label="val_acc")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("Unet_batchnorm.png")
plt.show()

In [ ]:
# Instantiate the model
model = U_NET(shape=(WINDOW_SIZE, WINDOW_SIZE, 3))

# Load the model
model.load("Unet_batchnorm-036-0.944867-0.913723.h5")

model.model.summary()

# We add all test images to an array, used later for generating a submission
image_filenames = []
for i in range(1, 51):
    image_filename = "data/test_set_images/test_" + str(i) + "/test_" + str(i) + ".png"
    image_filenames.append(image_filename)

# Set-up submission filename
submission_filename = "Unet_batchnorm-036-0.944867-0.913723.csv"

# Generates the submission
generate_submission(model, submission_filename, *image_filenames)